# GenAI-Powered Healthcare Assistant: Smarter Symptom Insights with Gemini

## Introduction
In a world where health concerns can arise at any moment, people often turn to the internet to make sense of their symptoms. Unfortunately, the answers they find are usually generic, overwhelming, or even misleading. What if there was a way to get personalized, conversational, and medically informed responses using the power of Generative AI?

This project leverages **Google's Gemini models** to create an intelligent healthcare assistant that can engage in natural dialogue, interpret symptoms, suggest possible diagnoses, and recommend next steps—all within seconds.

Target Audience: The primary audience for this **AI-Powered Healthcare Assistant** includes:

- Patients: Individuals seeking immediate, reliable information about their symptoms and potential health conditions.
- Healthcare Providers: Medical professionals looking for tools to enhance patient engagement and streamline the diagnostic process.
- Caregivers: Family members or friends assisting patients in managing health concerns and seeking appropriate care.

## Problem Statement
Traditional symptom checkers and online search results often lack personalization and contextual accuracy. Users are either bombarded with too much irrelevant information or given vague responses that leave them confused. There's a growing need for a smarter, more interactive system that provides **reliable and human-like health guidance** based on user inputs.



## Proposed Solution
We present a **GenAI-powered healthcare chatbot** that:
- Understands and processes natural language symptom descriptions,
- Identifies likely medical conditions,
- Provides treatment suggestions and preventive tips,
- Retrieves relevant medical knowledge from a custom-trained database (RAG),
- Responds in both human-friendly paragraphs and structured JSON formats for API-readiness.



## Techniques Used
- **Google Gemini 2.0 Flash API** for conversational AI and content generation.
- **Prompt Engineering**: Few-shot prompting for high-quality, domain-specific responses.
- **Config Tuning**: Fine control using temperature, top-p, and token limits.
- **Structured Output**: Enforcing JSON response schema using TypedDict.
- **Retrieval-Augmented Generation (RAG)**: Using ChromaDB and Gemini embeddings to pull in relevant medical knowledge based on symptoms.
- **SQL Integration**: Simulating a backend with SQLite for structured symptom-to-diagnosis mappings.
- **Interactive Chat**: Maintaining session-based chats for natural interaction.



This notebook blends **conversational AI**, **information retrieval**, and **structured data workflows** to deliver a robust prototype for intelligent virtual healthcare support.


In [1]:
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.9 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [3]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

Testing out the model by giving a sample prompt

In [5]:
# Sample Prompt
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="I am experiencing high temperature and coughing. What am I suffering with?")

print(response.text)

I am an AI and cannot provide medical advice. Experiencing a high temperature and coughing could be due to a number of reasons, including:

*   **Common Cold:** Usually accompanied by other symptoms like a runny nose, sore throat, and body aches.
*   **Flu (Influenza):** Similar to a cold but often more severe, with higher fever, body aches, and fatigue.
*   **COVID-19:** Can have a wide range of symptoms, including fever, cough, loss of taste or smell, and shortness of breath.
*   **Bronchitis:** Inflammation of the bronchial tubes, often causing a persistent cough.
*   **Pneumonia:** An infection of the lungs that can cause a high fever, cough, and difficulty breathing.

**It is important to consult with a healthcare professional or doctor for an accurate diagnosis and appropriate treatment. Self-treating can be dangerous.**


The output from the previous prompt is displayed as plain text. Consider formatting it properly using **Markdown** to enhance readability and user experience.

In [6]:
# Markdown format
Markdown(response.text)

I am an AI and cannot provide medical advice. Experiencing a high temperature and coughing could be due to a number of reasons, including:

*   **Common Cold:** Usually accompanied by other symptoms like a runny nose, sore throat, and body aches.
*   **Flu (Influenza):** Similar to a cold but often more severe, with higher fever, body aches, and fatigue.
*   **COVID-19:** Can have a wide range of symptoms, including fever, cough, loss of taste or smell, and shortness of breath.
*   **Bronchitis:** Inflammation of the bronchial tubes, often causing a persistent cough.
*   **Pneumonia:** An infection of the lungs that can cause a high fever, cough, and difficulty breathing.

**It is important to consult with a healthcare professional or doctor for an accurate diagnosis and appropriate treatment. Self-treating can be dangerous.**

We can also start a chat session using the Gemini model. It sends a simple message to begin the conversation. The chat keeps `history`, so it can remember what was said earlier in the conversation.


In [7]:
# Starting a chat session
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hey. Can I ask you a question?')
Markdown(response.text)

Yes, of course! Ask away. I'm ready to answer it if I can.


In [8]:
response = chat.send_message('I have been coughing from many days. How should I treat it?')
Markdown(response.text)

Okay, a persistent cough can be really annoying! Since it's been going on for many days, it's definitely a good idea to figure out what's causing it. **I am not a doctor, so I can't give you medical advice or diagnose you. The best thing to do is to see a doctor or other qualified healthcare professional to get a proper diagnosis and treatment plan.**

However, I can tell you some general information about coughs and what you can do while you wait to see a doctor, or to help manage symptoms in addition to your doctor's recommendations.

**Things a doctor will consider:**

*   **How long have you had the cough?** This helps determine if it's acute (short-term) or chronic (long-term).
*   **What kind of cough is it?** Is it dry, productive (bringing up mucus), hacking, whooping, etc.?
*   **What are your other symptoms?** Fever, sore throat, runny nose, chest pain, shortness of breath, fatigue, etc.?
*   **Do you have any underlying health conditions?** Asthma, COPD, allergies, etc.?
*   **Are you taking any medications?** Some medications can cause a cough as a side effect.
*   **Do you smoke or are you exposed to secondhand smoke?**
*   **Have you been exposed to any irritants or allergens?**

**Potential Causes:**

Coughs can be caused by a variety of things, including:

*   **Common Cold/Flu:** Usually accompanied by other symptoms like runny nose, sore throat, and fever.
*   **Bronchitis:** Inflammation of the bronchial tubes, often causing a productive cough.
*   **Pneumonia:** Infection of the lungs, often causing a cough with phlegm, fever, and chest pain.
*   **Allergies:** Can cause a dry cough, especially during allergy season.
*   **Asthma:** Can cause coughing, wheezing, and shortness of breath.
*   **GERD (Gastroesophageal Reflux Disease):** Stomach acid refluxing into the esophagus can irritate the throat and cause a cough.
*   **Postnasal Drip:** Mucus draining down the back of the throat can trigger a cough.
*   **Whooping Cough (Pertussis):** A highly contagious bacterial infection that causes severe coughing fits.
*   **COPD (Chronic Obstructive Pulmonary Disease):** A lung disease that causes airflow obstruction and can lead to chronic cough.
*   **Smoking:** A common cause of chronic cough.
*   **ACE Inhibitors:** Certain blood pressure medications can cause a dry cough.

**What you can do in the meantime (for symptom relief, *not* as a replacement for medical advice):**

*   **Stay Hydrated:** Drink plenty of fluids, especially water, to help thin mucus.
*   **Honey:** A spoonful of honey can help soothe a cough, especially for children over 1 year old. (Do not give honey to infants under 1 year old due to the risk of botulism.)
*   **Steam:** Take a hot shower or use a humidifier to moisten the air and loosen congestion.
*   **Rest:** Get plenty of rest to help your body heal.
*   **Over-the-Counter Medications:**
    *   **Cough Suppressants (e.g., Dextromethorphan):** Can help reduce the urge to cough, especially for dry coughs.
    *   **Expectorants (e.g., Guaifenesin):** Can help thin mucus and make it easier to cough up.
    *   **Decongestants (e.g., Pseudoephedrine, Phenylephrine):** Can help relieve nasal congestion, which can contribute to postnasal drip.  **Be careful with decongestants if you have high blood pressure or heart problems.**
    *   **Pain Relievers (e.g., Acetaminophen, Ibuprofen):** Can help relieve fever and body aches.
*   **Avoid Irritants:** Stay away from smoke, dust, and other irritants that can worsen your cough.
*   **Elevate Your Head:** Sleeping with your head elevated can help reduce postnasal drip and ease coughing at night.
*   **Salt Water Gargle:** Gargling with warm salt water can soothe a sore throat.

**When to see a doctor *immediately*:**

*   **Difficulty breathing or shortness of breath.**
*   **Chest pain.**
*   **Coughing up blood.**
*   **High fever (over 103°F or 39.4°C).**
*   **Bluish lips or face.**
*   **Severe weakness or dizziness.**
*   **If you have a chronic medical condition like asthma or COPD and your cough is getting worse.**
*   **If your cough lasts for more than 3 weeks.**
*   **If you are concerned about your symptoms.**

**In summary, while there are things you can do to help manage your cough, it's really important to see a doctor to get a diagnosis and rule out any serious underlying conditions, especially since it's been going on for days. Don't delay seeking professional medical advice.**

I hope this information is helpful. Good luck, and I hope you feel better soon! Remember to consult with a healthcare professional for proper diagnosis and treatment.


In [9]:
response = chat.send_message('What is the symptom that i mentioned earlier?')
Markdown(response.text)

You mentioned experiencing a **cough for many days**. That's the symptom you initially described.


# **Customizing Gemini AI for Healthcare Assistance**

To ensure the responses generated by the healthcare assistant are aligned with the specific requirements of the application, it is often necessary to adjust the model's configuration. Gemini AI offers several powerful features that support such customization, enabling control over the tone, creativity, focus, and length of the output.

These configurable parameters allow the system to produce context-aware and user-friendly responses—an essential aspect when dealing with healthcare-related queries.

Some of the key features available for customization include:


### **Max Output Tokens**
In the context of a healthcare assistant, this parameter can be adjusted based on the desired level of detail. Lower values may be suitable for brief suggestions or symptom confirmations, while higher values are useful for more detailed explanations or structured outputs like treatment plans.


In [10]:
# Output length 
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=100)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Give me remedies to treat cold and cough')

Markdown(response.text)

Okay, let's explore some remedies for treating a cold and cough.  It's important to remember that I am an AI and cannot provide medical advice.  If your symptoms are severe, persistent, or you have underlying health conditions, **please consult with a doctor or other qualified healthcare professional.**

**Home Remedies & Supportive Care:**

*   **Rest:**  This is crucial.  Your body needs energy to fight off the virus. Get plenty of sleep and avoid strenuous activity.

### **Temperature** 
For a healthcare assistant, where responses often need to be clear, consistent, and fact-based, lower temperature values are generally preferred. These produce more deterministic and focused outputs. In contrast, higher temperatures introduce more variation and creativity, which may be useful for generating broad wellness advice or alternative suggestions.

A temperature of `0` results in highly predictable outputs, selecting the most likely tokens every time, while higher values allow for a wider range of responses.

In [11]:
# Temperatures
low_temp_config = types.GenerateContentConfig(temperature=0.0)
high_temp_config = types.GenerateContentConfig(temperature=2.0)

In [12]:
# Configuring with low temperature 
for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=low_temp_config,
      contents='Give me a name of a disease in random in a single word')

  if response.text:
    print(response.text, '-' * 25)

Pneumonia
 -------------------------
Pneumonia
 -------------------------
Pneumonia
 -------------------------
Pneumonia
 -------------------------
Pneumonia
 -------------------------


In [13]:
# Configuring with high temperature 
for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=high_temp_config,
      contents='Give me a name of a disease in random in a single word')

  if response.text:
    print(response.text, '-' * 25)

Gout
 -------------------------
Tuberculosis
 -------------------------
Tuberculosis
 -------------------------
Measles
 -------------------------
Eczema
 -------------------------


### **Top-p**
Top-p is used to define the randomess of tokens selected by a model.

In [14]:
# Response with lower randomess
top_p_config = types.GenerateContentConfig(top_p=0.1, max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=top_p_config,
    contents='Describe COVID-19 in a paragraph')

Markdown(response.text)

COVID-19, short for Coronavirus Disease 2019, is an infectious disease caused by the SARS-CoV-2 virus. Primarily spread through respiratory droplets produced when an infected person coughs, sneezes, talks, or breathes, it can cause a wide range of symptoms, from mild cold-like symptoms such as fever, cough, and fatigue, to severe illness including pneumonia, acute respiratory distress syndrome, and even death. The virus has led to a global pandemic, causing widespread disruption to healthcare systems, economies, and daily life, and prompting the development and deployment of vaccines and treatments to mitigate its impact.


In [15]:
# Response with higher randomness
top_p_config = types.GenerateContentConfig(top_p=1.0, max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=top_p_config,
    contents='Describe COVID-19 in a paragraph')

Markdown(response.text)

COVID-19, short for Coronavirus Disease 2019, is a contagious respiratory illness caused by the SARS-CoV-2 virus.  It primarily spreads through respiratory droplets produced when an infected person coughs, sneezes, talks, or breathes. Symptoms range from mild, like fever, cough, and fatigue, to severe, including pneumonia, acute respiratory distress syndrome, and even death. The virus can affect various organs, leading to long-term health issues. While vaccines and treatments have reduced the severity and spread, COVID-19 remains a global health concern requiring ongoing monitoring, vaccination efforts, and preventative measures.


### **Few Shot Prompting**
The function `healthcare_assistant` in the code leverages Few-shot Prompting to guide the language model to process symptoms and generate medical insights in a specific format (JSON in this case). 
By providing a small number of example inputs and their corresponding outputs, the model can understand how to respond to new, unseen inputs in a similar manner. The output JSON includes:

- The symptoms extracted from the input.

- Possible conditions based on the symptoms.

- Relevant medical advice or recommendations.

In [16]:
# Few-shot Prompting
def healthcare_assistant(symptoms):
    # Few-shot examples included in the prompt
    few_shot_prompt = """Parse a user's symptoms into valid JSON for medical insights:

    EXAMPLE:
    I have a fever and sore throat.
    JSON Response:
    ```
    {
        "symptoms": ["fever", "sore throat"],
        "possible_conditions": ["Common Cold","Flu"],
        "recommendations": [
            "Stay hydrated and rest.",
            "Use lozenges to soothe your throat.",
            "Consult a doctor if symptoms persist for more than three days."
        ]
    }
    ```

    EXAMPLE:
    I'm experiencing chest pain and shortness of breath.
    JSON Response:
    ```
    {
        "symptoms": ["chest pain", "shortness of breath"],
        "possible_conditions": ["Heart Disease","Panic Attack"],
        "recommendations": [
            "Seek immediate medical attention.",
            "Avoid stressful activities.",
            "Schedule a consultation with a cardiologist."
        ]
    }
    ```

    USER SYMPTOMS:
    """
    response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, symptoms])

    return response.text

# Example Usage
customer_symptoms = "I feel nauseous and have a headache."
output = healthcare_assistant(customer_symptoms)
Markdown(output)

```json
{
    "symptoms": ["nausea", "headache"],
    "possible_conditions": ["Migraine", "Food Poisoning", "Dehydration"],
    "recommendations": [
        "Stay hydrated.",
        "Rest in a quiet, dark room.",
        "Take over-the-counter pain relievers as directed.",
        "If symptoms worsen or persist, consult a doctor."
    ]
}
```

Few-shot prompting applies examples of text to direct the response of the model while JSON schema applies to impose a strict, structured response to ensure that the output corresponds to the specified format

In [17]:
# JSON Mode
import typing_extensions as typing

class HealthcareAssistent(typing.TypedDict):
    symptoms: list[str]
    possible_conditions: list[str]
    recommendations: list[str]


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.15, # degree of randomeness
        top_p=0.5, # number of tokens used
        response_mime_type="application/json",
        response_schema=HealthcareAssistent,
    ),
    contents="I have persistent fatigue and muscle pains.")

print(response.text)

{
  "symptoms": ["persistent fatigue", "muscle pains"],
  "possible_conditions": ["fibromyalgia", "chronic fatigue syndrome", "vitamin D deficiency", "hypothyroidism"],
  "recommendations": ["Consult a doctor for diagnosis and treatment", "Consider blood tests to check vitamin levels and thyroid function", "Explore pain management strategies", "Prioritize rest and stress reduction"]
}


# RAG (Retrieval-Augmented Generation) and Its Role in AI-Powered Healthcare Assistant

**RAG (Retrieval-Augmented Generation)** is a method based on the integration of **information retrieval** and **generative models** to enhance responsiveness and relevance of the response. The operation is carried out by initially retrieving associated data or documents from a query and then producing an answer by integrating the retrieved content along with the response abilities of the model. This method enhances the answers to a richer quality with more context so that the resulting answers are well-informed and correct where alone, the model's training would not be able to capture every subtlety.

### Why use RAG for healthcare assistant?
RAG is useful to an AI-based health assistant by facilitating dynamic information retrieval from the outside world, e.g., medical stores or files. Upon receiving user input in terms of symptoms or medication requests, the assistant queries against the knowledge store, retrieves pertinent data, and builds contextual correct answers. Through this activity, the assistant provides up-to-date as well as accurate information, enhancing user satisfaction and facilitating sound decision-making.

I utilized **ChromaDB** as the vector database to store and retrieve pertinent medical documents. Upon a user submitting a query (e.g., "What do I do if I am nauseous after eating peanuts?"), the system first queries the database for pertinent documents using the **embedding function**.
Once the system identifies the most appropriate document(s), it merges the information with the generative model to create a response exclusively for the user's question.

### Data Source
The data for this project was sourced into **11 small medical documents** available on the official NHS website [Illnesses and conditions](https://www.nhsinform.scot/illnesses-and-conditions/)  
These documents contain information about various medical conditions, symptoms, and treatments. 

In [18]:
data="""A cold is a common mild viral infection that typically affects the nose and throat and usually resolves within 1 to 2 weeks. Symptoms include a sore throat, blocked or runny nose, sneezing, cough, hoarse voice, and general fatigue, while less frequent symptoms may involve fever, headache, earache, muscle aches, mild eye irritation, and loss of taste and smell. Remedies to alleviate symptoms include getting plenty of rest, staying hydrated, eating a balanced diet, taking over-the-counter pain relief for fever and aches, using decongestants for nasal congestion, and trying home remedies like gargling salt water or sucking menthol sweets. Preventive measures include regular handwashing, coughing or sneezing into tissues, and avoiding sharing utensils, towels, or toys to reduce the spread.

Flu, or influenza, is a common viral illness that spreads easily, particularly during the winter months. It often comes on quickly with symptoms such as a sudden high fever, tiredness, headaches, general aches, a dry cough, sore throat, and chills. Additional symptoms can include loss of appetite, nausea, vomiting, diarrhea, and nasal congestion. Most people recover within a week, though fatigue and a lingering cough may persist longer. To alleviate symptoms, rest, stay hydrated, keep warm, and take over-the-counter medications like paracetamol or ibuprofen for fever and aches. Preventive measures include regular handwashing, covering your mouth and nose when sneezing or coughing, cleaning commonly touched surfaces, avoiding close contact with others while infectious, and getting the annual flu vaccine to stay protected.

Allergies occur when the body's immune system reacts to certain substances, known as allergens, as if they are harmful. Allergies are common, affecting over 1 in 4 people in the UK, and they can develop at any age. Common allergens include pollen, dust mites, animal dander, certain foods like nuts and shellfish, insect stings, medications, latex, mould, and household chemicals. Symptoms of an allergic reaction usually appear quickly and can include sneezing, a runny or blocked nose, red and itchy eyes, wheezing, coughing, skin rashes, and worsened asthma or eczema. Severe reactions, such as anaphylaxis, require urgent medical treatment. Allergies can often be managed by avoiding allergens and using treatments like antihistamines, decongestants, creams, and steroids. For severe cases, immunotherapy may be recommended to gradually reduce sensitivity to allergens. Preventing allergic reactions involves maintaining good hygiene, reading food labels, and minimizing exposure to known allergens.

High blood pressure (hypertension) is a condition where the force of blood against the walls of the arteries is consistently too high, increasing the risk of cardiovascular diseases like heart attack or stroke. It is typically diagnosed when blood pressure readings are regularly above 140/90 mmHg. Common risk factors include a high-fat or high-salt diet, lack of exercise, obesity, smoking, excessive alcohol intake, stress, certain medications, and underlying conditions like kidney disease or diabetes. Hypertension often has no symptoms, but in rare cases, it may cause headaches, blurred vision, nosebleeds, or shortness of breath. Managing high blood pressure involves regular exercise, a healthy low-salt diet, reduced alcohol and caffeine intake, weight loss if necessary, and relaxation techniques such as yoga. Medications such as ACE inhibitors, calcium channel blockers, or beta-blockers may be prescribed and might need to be taken long-term. Monitoring blood pressure regularly is essential to ensure effective treatment and lifestyle adjustments.

Arthritis is a common condition that causes pain and inflammation in the joints, affecting people of all ages, including children. The two most common types are osteoarthritis and rheumatoid arthritis, but other types include gout, ankylosing spondylitis, and psoriatic arthritis. Symptoms vary depending on the type but generally include joint pain, tenderness, stiffness, inflammation, restricted movement, weakness, and warm, red skin around the joints. While there is no cure, treatments like painkillers, NSAIDs, corticosteroids, disease-modifying anti-rheumatic drugs (DMARDs), physiotherapy, and regular exercise can help manage the condition. In severe cases, surgery such as joint replacement or fusion may be necessary. Protecting joints through proper posture, using supportive tools, and making ergonomic adjustments at home can reduce strain and prevent further damage. Occupational therapy can assist with maintaining independence in daily tasks, and a tailored approach combining treatments is often the most effective way to manage arthritis.

Migraine is a common health condition that often begins in early adulthood, affecting around 1 in 5 women and 1 in 15 men. It is characterized by a throbbing headache, typically on one side of the head, which may occur frequently or sporadically over years. Symptoms can include nausea, vomiting, sensitivity to light and sound, sweating, dizziness, and abdominal pain, often lasting between 4 hours to 3 days, with some people also experiencing aura, such as visual disturbances or tingling sensations, before the headache. Triggers can include hormonal changes, stress, fatigue, dietary factors like caffeine or chocolate, dehydration, and environmental stimuli like bright lights or loud sounds. Though the exact cause is unknown, it is linked to abnormal brain activity and may have a genetic component. Managing migraines involves identifying triggers, maintaining a healthy lifestyle, and using treatments like painkillers or prescribed medications. Proper diagnosis by a GP is essential to rule out other conditions and determine the appropriate course of action.

Irritable bowel syndrome (IBS) is a common, long-term condition of the digestive system that can persist throughout life, though symptoms may change over time. It does not pose a serious risk to physical health and is not linked to cancer or other bowel-related diseases. The exact cause is unknown, but symptoms such as abdominal pain or cramping, changes in bowel habits (diarrhoea or constipation), bloating, and excessive wind can vary in severity and often worsen with stress or certain foods. Less common symptoms include lethargy, nausea, and heartburn. IBS can impact daily life and may lead to stress or low mood. Diagnosis typically involves symptom history and tests to rule out other conditions like coeliac disease. Management strategies include working with a GP to identify triggers and avoid dietary changes until necessary testing is completed. While IBS can be managed with the right approach, further symptoms such as unexplained weight loss, bleeding, or lumps should prompt urgent medical evaluation.

Urinary tract infections (UTIs) are common infections affecting the bladder, kidneys, and related tubes, with women more likely to experience them. UTIs can cause symptoms like frequent or painful urination, cloudy or foul-smelling urine, and lower abdominal pain, while upper UTIs may include fever, back pain, and nausea. Most UTIs are treated with antibiotics and improve within 3 to 5 days, though chronic UTIs may require longer treatments. Preventive measures include staying hydrated, wiping from front to back, emptying the bladder fully, and avoiding irritants like perfumed soaps. If symptoms worsen or don’t improve, consulting a GP is crucial, as untreated UTIs can lead to serious complications like kidney infections or sepsis.

Obesity is a condition characterized by excessive body fat that can lead to serious health issues, affecting around one in four adults and one in five children in the UK. It is typically measured using Body Mass Index (BMI), with a BMI of 30 or above classified as obese. A waist circumference over 94 cm in men and 80 cm in women can further indicate obesity-related health risks, including type 2 diabetes, coronary heart disease, certain cancers, and stroke. Obesity is primarily caused by consuming more calories than are burned through physical activity, often due to modern lifestyles involving high-calorie diets and sedentary habits. Treatment involves adopting a calorie-controlled diet, engaging in regular exercise, and seeking psychological support to change eating behaviors. In some cases, medication like orlistat or, rarely, weight loss surgery may be recommended. Preventing and addressing obesity is essential to improving overall health and quality of life.

Myalgic encephalomyelitis (ME), also known as chronic fatigue syndrome (CFS), is a long-term neurological condition affecting the nervous and immune systems, causing severe fatigue and pain that doesn’t improve with sleep or rest. A hallmark symptom is post-exertional malaise (PEM), where even minimal activity leads to debilitating exhaustion, which might appear days later. Other symptoms include broken sleep, brain fog, dizziness, nausea, sensitivity to light and sound, and muscle pain. The condition affects individuals of all ages, genders, and backgrounds and can vary in severity, with periods of improvement and worsening. While the exact cause is unknown, potential triggers include infections, trauma, or major life stressors. Diagnosis requires ruling out other conditions, often through symptom tracking and routine tests. Though there is no cure, management strategies such as pacing, dietary advice, medication, counselling, and cognitive behavioural therapy (CBT) can help alleviate symptoms and improve quality of life. It is crucial to work with a GP to create a tailored approach for symptom management.

A fever is typically identified when body temperature rises to 38°C or higher, often accompanied by sensations of warmth, chills, or shivering. It is the body’s natural response to infections such as flu, tonsillitis, or urinary tract infections (UTIs), helping to stimulate the immune system and hinder the survival of bacteria and viruses. Most fevers resolve within a few days and can be managed by resting, staying hydrated, wearing comfortable clothing, and taking fever-reducing medications like paracetamol. It’s important to avoid over-layering or attempting to cool down forcefully, as this may worsen symptoms. Seek medical advice if the fever persists, worsens, or is accompanied by severe symptoms such as confusion, muscle cramps, or unusual thirst. Fever, although uncomfortable, plays an essential role in fighting infections."""

documents =data.split('\n\n')
print(f"Number of documents: {len(documents)}")

Number of documents: 11


In [19]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.7 MB/s eta 0:

In [20]:
# Creating a Gemini Embedding Function
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.genai import types


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]


The code initializes a Chroma database named `googlecardb`, sets up an embedding function, and creates a collection. It then adds the documents to the database, assigning unique IDs to each document.

In [21]:
# Initializing an empty database
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

/tmp/ipykernel_13/4292023148.py:6: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embed_fn = GeminiEmbeddingFunction()


Confirm whether the data has been successfully inserted into the database.

In [22]:
# Confirming whether the data has been successfully inserted into the database
db.count()

11

In [23]:
# Query1 to the database
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "I am experiencing high temperature and tiredness.How should i treat it?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

A fever is typically identified when body temperature rises to 38°C or higher, often accompanied by sensations of warmth, chills, or shivering. It is the body’s natural response to infections such as flu, tonsillitis, or urinary tract infections (UTIs), helping to stimulate the immune system and hinder the survival of bacteria and viruses. Most fevers resolve within a few days and can be managed by resting, staying hydrated, wearing comfortable clothing, and taking fever-reducing medications like paracetamol. It’s important to avoid over-layering or attempting to cool down forcefully, as this may worsen symptoms. Seek medical advice if the fever persists, worsens, or is accompanied by severe symptoms such as confusion, muscle cramps, or unusual thirst. Fever, although uncomfortable, plays an essential role in fighting infections.

In [24]:
# Query2 to the database
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "What should I do if i am experiencing uneasiness after eating peanuts?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Allergies occur when the body's immune system reacts to certain substances, known as allergens, as if they are harmful. Allergies are common, affecting over 1 in 4 people in the UK, and they can develop at any age. Common allergens include pollen, dust mites, animal dander, certain foods like nuts and shellfish, insect stings, medications, latex, mould, and household chemicals. Symptoms of an allergic reaction usually appear quickly and can include sneezing, a runny or blocked nose, red and itchy eyes, wheezing, coughing, skin rashes, and worsened asthma or eczema. Severe reactions, such as anaphylaxis, require urgent medical treatment. Allergies can often be managed by avoiding allergens and using treatments like antihistamines, decongestants, creams, and steroids. For severe cases, immunotherapy may be recommended to gradually reduce sensitivity to allergens. Preventing allergic reactions involves maintaining good hygiene, reading food labels, and minimizing exposure to known allergens.

### Approach for Query Processing and Answer Generation:

The technique starts by preprocesses the query of the user into a clear, single-line input. Then, a framed prompt is constructed, directing the AI model to give a crisp, informative, and well-padded response. The prompt further contains directives on highlighting the illness, offering solutions, and suppressing irrelevant content. Pertinent passages of medical material retrieved from the vector database are added to the prompt to bolster contextual understanding.

Finally, the prompt is submitted to the Gemini model (`gemini-2.0-flash`) which returns a context-specific response as per both the query of the user and reference documents. Then, the output is presented in clean Markdown-readable form so the answer is easily comprehensible.

In [25]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Make sure to give the answer in a simple and understandable format.The response should be in a paragraph but in a proper format. 
The illness should be highlighted with proper remedies.
If the passage is irrelevant to the answer, you may ignore it.
{query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Make sure to give the answer in a simple and understandable format.The response should be in a paragraph but in a proper format. 
The illness should be highlighted with proper remedies.
If the passage is irrelevant to the answer, you may ignore it.
What should I do if i am experiencing uneasiness after eating peanuts?
PASSAGE: Allergies occur when the body's immune system reacts to certain substances, known as allergens, as if they are harmful. Allergies are common, affecting over 1 in 4 people in the UK, and they can develop at any age. Common allergens include pollen, dust mites, animal dander, certain foods like nuts and shellfish, insect stings, medications, latex, mould, and household chemicals. Symptoms of an allergic reaction usually appear quickly and can include sneezing, a runny or blocked nose, red and itchy eyes, wheezing, coughing, skin rashes, and worsened a

In [26]:
# Generating in a proper format
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

If you are experiencing uneasiness after eating peanuts, it is possible you are having an allergic reaction. Allergies to foods like nuts are common and can cause symptoms that appear quickly. The passage mentions that treatments like antihistamines, decongestants, creams, and steroids can help manage allergies. In severe cases, immunotherapy may be recommended to gradually reduce sensitivity to allergens. To prevent future reactions, it is important to read food labels carefully to avoid peanuts and maintain good hygiene to minimize exposure to allergens. If the reaction is severe, seek urgent medical treatment.


# **Function Calling using SQLite database**

In this AI-powered healthcare assistant project, **function calling with SQLite** was used to enable intelligent, database-backed responses to user queries. A local SQLite database named `aihealth.db` was created, and **two main tables** were designed:

- `Symptoms` – stores symptom descriptions and severity levels.  
- `Diagnoses` – contains corresponding diagnoses, treatments, medications, dosages, and side effects linked to symptoms.

Using Python’s `sqlite3` module and SQL commands, these tables were populated with relevant medical data.

To allow the Gemini model to interact with the database, **function calling was set up** with:
- `list_tables()` – lists all available tables,
- `display_table(table_name)` – shows table data,
- `execute_query(query)` – runs custom SQL queries.

In [27]:
%load_ext sql
%sql sqlite:///aihealth.db

In [28]:
 %reload_ext sql

Two tables, Symptoms and Diagnoses are created

In [29]:
%%sql
-- Creating the Symptoms Table
CREATE TABLE Symptoms (
    symptom_id INTEGER PRIMARY KEY AUTOINCREMENT,
    symptom_description TEXT NOT NULL,
    severity TEXT
);

-- Creating the Diagnoses Table
CREATE TABLE Diagnoses (
    diagnosis_id INTEGER PRIMARY KEY AUTOINCREMENT,
    symptom_id INTEGER NOT NULL,
    diagnosis TEXT NOT NULL,
    treatment_description TEXT, -- Treatment added directly to Diagnoses
    medication_name TEXT,       -- Medication added directly to Diagnoses
    dosage TEXT,                -- Dosage for the medication
    side_effects TEXT,          -- Side effects of the medication
    FOREIGN KEY (symptom_id) REFERENCES Symptoms (symptom_id)
);



 * sqlite:///aihealth.db
Done.
Done.


[]

Once the tables are created, the tables are populated with relevant data

In [30]:
%%sql
-- Inserting data into Symptoms Table
INSERT INTO Symptoms (symptom_description, severity) VALUES 
('Sore throat and runny nose', 'Mild'),
('Fever and body aches', 'Moderate'),
('Sneezing and itchy eyes', 'Mild'),
('Headache and blurred vision', 'Moderate'),
('Joint pain and stiffness', 'Severe'),
('Throbbing headache and nausea', 'Moderate'),
('Abdominal pain and bloating', 'Mild'),
('Frequent urination and pain while urinating', 'Severe'),
('Excess weight and fatigue', 'Moderate'),
('Fatigue and dizziness', 'Severe'),
('High temperature and chills', 'Moderate'),
('Shortness of breath and wheezing', 'Severe');

-- Inserting data into Diagnoses Table
INSERT INTO Diagnoses (symptom_id, diagnosis, treatment_description, medication_name, dosage, side_effects) VALUES 
(1, 'Cold', 'Stay hydrated and rest.', 'Decongestant', '2 tablets daily', 'Drowsiness'),
(2, 'Flu', 'Take paracetamol and drink fluids.', 'Paracetamol', '500 mg every 6 hours', 'None'),
(3, 'Allergies', 'Use antihistamines and avoid allergens.', 'Antihistamine', '1 tablet daily', 'Dry mouth'),
(4, 'High Blood Pressure', 'Monitor blood pressure and take prescribed medication.', 'Amlodipine', '5 mg daily', 'Dizziness'),
(5, 'Arthritis', 'Apply physiotherapy and take anti-inflammatory drugs.', 'Ibuprofen', '200 mg every 8 hours', 'Stomach upset'),
(6, 'Migraine', 'Use pain relievers and avoid bright light.', 'Sumatriptan', '50 mg as needed', 'Tingling sensation'),
(7, 'Irritable Bowel Syndrome', 'Make dietary changes and reduce stress.', 'Probiotic Supplement', '1 capsule daily', 'None'),
(8, 'Urinary Tract Infection', 'Take antibiotics and drink plenty of water.', 'Nitrofurantoin', '100 mg twice daily', 'Nausea'),
(9, 'Obesity', 'Adopt a low-calorie diet and exercise regularly.', 'Orlistat', '120 mg with meals', 'Abdominal discomfort'),
(10, 'Chronic Fatigue Syndrome', 'Practice energy pacing and seek therapy.', 'Modafinil', '200 mg daily', 'Headache'),
(11, 'Fever', 'Take fever-reducing medication and rest.', 'Ibuprofen', '200 mg every 8 hours', 'Stomach upset'),
(12, 'Asthma', 'Use inhalers and practice breathing exercises.', 'Salbutamol Inhaler', '2 puffs every 4 hours', 'Mild throat irritation');

 * sqlite:///aihealth.db
12 rows affected.
12 rows affected.


[]

Connection to a local SQLite database is established so the AI assistant can interact with and query health-related data. This enables retrieval of relevant information like symptoms, diagnoses, and treatments for user queries.

In [31]:
# Creating a connection
import sqlite3

db_file = "aihealth.db"
db_conn = sqlite3.connect(db_file)

In [32]:
# Displaying the names of the tables 
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print('- DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]

print('List of Tables created: ')
list_tables()

List of Tables created: 
- DB CALL: list_tables()


['Symptoms', 'sqlite_sequence', 'Diagnoses']

In [33]:
# Displaying the data in a table
def display_table(table_name: str) -> list[tuple]:
    """Retrieve all the rows from a specific table"""
    print(f'Display table {table_name}')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute(f"SELECT * FROM {table_name};")

    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print()

display_table("Symptoms")


Display table Symptoms
(1, 'Sore throat and runny nose', 'Mild')
(2, 'Fever and body aches', 'Moderate')
(3, 'Sneezing and itchy eyes', 'Mild')
(4, 'Headache and blurred vision', 'Moderate')
(5, 'Joint pain and stiffness', 'Severe')
(6, 'Throbbing headache and nausea', 'Moderate')
(7, 'Abdominal pain and bloating', 'Mild')
(8, 'Frequent urination and pain while urinating', 'Severe')
(9, 'Excess weight and fatigue', 'Moderate')
(10, 'Fatigue and dizziness', 'Severe')
(11, 'High temperature and chills', 'Moderate')
(12, 'Shortness of breath and wheezing', 'Severe')



In [34]:
def execute_query(query: str) -> list[tuple]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({query})')

    cursor = db_conn.cursor()

    cursor.execute(query)
    return cursor.fetchall()


execute_query("select diagnosis from Diagnoses")

 - DB CALL: execute_query(select diagnosis from Diagnoses)


[('Cold',),
 ('Flu',),
 ('Allergies',),
 ('High Blood Pressure',),
 ('Arthritis',),
 ('Migraine',),
 ('Irritable Bowel Syndrome',),
 ('Urinary Tract Infection',),
 ('Obesity',),
 ('Chronic Fatigue Syndrome',),
 ('Fever',),
 ('Asthma',)]

### Enabling the assistant to Interact with a Healthcare Database

The healthcare assistant powered by AI is intended to reply to user queries through an interaction with an underlying database. The methodology involves:

- Functions `list_tables`, `display_table`, and `execute_query` are made available as tools. These enable the GenAI model to traverse database schema and retrieve data based on user queries.

- A well-defined system instruction is given to the model, instructing it to translate user health-related queries to SQL queries, utilize the tools to pull data, and output in a plain and useful manner.

This facilitates the smooth conversational interface whereby the model is able to conduct real-time lookups (such as treatments, medications, or side effects) in real-time from the database via function calling.

In [35]:
# Define the database interaction tools
db_tools = [list_tables, display_table, execute_query]

# Instruction for the AI-powered healthcare assistant
instruction = """You are a helpful chatbot designed to interact with an SQL database
for healthcare management. Your role is to assist users by answering their health-related 
questions using the information stored in the database. You will turn the users' questions 
into SQL queries using the tools available. After retrieving the relevant data, you will 
interpret it to provide meaningful answers in an easy-to-understand way.

Use list_tables to see what tables are present, display_table to see what data is present 
in it, and execute_query to issue an SQL SELECT query.

Tha table Diagnoses contains the columns: symptom_id, diagnosis, treatment_description, medication_name, dosage, side_effects."""""

# Initializing the GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat session with automatic function calling enabled
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [36]:
response = chat.send_message("What are the treatments and medications recommended for fever?")
print(response.text)

 - DB CALL: execute_query(SELECT treatment_description, medication_name FROM Diagnoses WHERE diagnosis = 'Fever')
The recommended treatment for fever is to take fever-reducing medication and rest. The suggested medication is Ibuprofen.



In [37]:
response = chat.send_message("What are the recommended treatments for Arthritis?")
print(response.text)

 - DB CALL: execute_query(SELECT treatment_description FROM Diagnoses WHERE diagnosis = 'Arthritis')
The recommended treatment for Arthritis is physiotherapy and taking anti-inflammatory drugs.



In [38]:
response = chat.send_message("What are the side effects of the medication Modafinil?")
print(response.text)

 - DB CALL: execute_query(SELECT side_effects FROM Diagnoses WHERE medication_name = 'Modafinil')
The side effect of Modafinil is headache.



### **AI-Powered Healthcare Assistant**

This AI healthcare assistant connects to a SQLite database (`aihealth.db`) to provide accurate health-related information. 
It uses tools to query data from interconnected tables on symptoms and diagnoses. 
With GenAI integration, it converts user queries into SQL commands, fetches details like treatments and side effects, and presents clear, actionable responses. Its design ensures efficiency, adaptability, and user-friendly interaction, enhancing healthcare accessibility and diagnostics. Let me know if you'd like more details!

The healthcare assistant:
- Converts user requests concerned with healthcare into SQL query commands with the purpose of retrieving suitable information from a database. 
- Calls the database through functions such as `list_tables`, `display_table`, and `execute_query` in a way that health data can be retrieved and processed easily.
- Joins corresponding tables, i.e., `Symptoms` and `Diagnoses`, and matches symptoms with equivalent diagnoses, treatments, drugs, and side effects. 


In [39]:
import sqlite3
from google import genai
from google.genai import types
from typing import List, Dict, Any
from pydantic import BaseModel

In [40]:
# Database setup
db_file = "aihealth.db"
db_conn = sqlite3.connect(db_file)

This virtual healthcare assistant interacts with an **SQL database** to help users with health-related inquiries. It converts user questions into SQL queries, retrieves data like **symptoms**, **diagnoses**, **treatments**, and **medication details**, and provides user-friendly responses by joining relevant database tables. 

Designed to ensure accuracy and accessibility, it operates seamlessly using defined tools for listing tables, displaying data, and executing queries. Let me know if you'd like to explore anything specific further!

In [41]:
# Define the database interaction tools
db_tools = [list_tables, display_table, execute_query]

# Instruction for the AI-powered healthcare assistant
instruction = """
You are a virtual healthcare assistant designed to interact with an SQL database for effective healthcare management. Your primary role is to help users by answering their health-related questions using the information stored in the database.

Your Responsibilities:
Query Interpretation: Convert user questions into SQL queries using the available tools.
Data Retrieval: Access the database to retrieve relevant information based on the queries you generate.
User -Friendly Responses: Interpret the retrieved data and provide clear, meaningful answers that are easy for users to understand.
Available Tools:
list_tables: Use this tool to display the names of all tables in the database.
display_table(table_name): Use this tool to view all data within a specific table.
execute_query(query): Use this tool to run an SQL SELECT query and obtain specific information.
Database Structure:
The database contains a table named Symptoms and Diagnoses with the following columns:

The table Diagnoses includes:
symptom_id: Unique identifier for each symptom.
diagnosis: The medical diagnosis associated with the symptoms.
treatment_description: Recommended treatment for the diagnosis.
medication_name: Name of the medication prescribed.
dosage: Dosage instructions for the medication.
side_effects: Possible side effects of the medication.

The table Symptoms includes:
symptom_description: description about the symptoms
severity: severity of the symptoms

To provide comprehensive answers, you will often need to join the Symptoms and Diagnoses tables. 
This allows you to correlate symptoms with their respective diagnoses, treatments, and medications. 
For example, you can retrieve a diagnosis and its associated treatment by joining these tables on the symptom_id.
"""

# Initializing the GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat session with automatic function calling enabled
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
        temperature=0.3,
        top_p=0.9,
    ),
)

The function `start_chat` starts a chatbot session, takes user input and generates responses.

In [42]:
import random
# Interactive chat session
start_prompt=["How can I support your health today?","I'm here to help—what would you like to know?","How can I assist you today?"," Just ask your question!","What health concern would you like to talk about?"]

def start_chat(user_input):
    print(random.choice(start_prompt))
    print("User: "+user_input)
    response = chat.send_message(user_input)
    print(response.text)

In [43]:
prompt="I am having joint pain. How should I treat it?"
start_chat(prompt)

What health concern would you like to talk about?
User: I am having joint pain. How should I treat it?
 - DB CALL: execute_query(SELECT t1.treatment_description FROM Diagnoses t1 JOIN Symptoms t2 ON t1.symptom_id = t2.symptom_id WHERE t2.symptom_description LIKE '%joint pain%')
Apply physiotherapy and take anti-inflammatory drugs.



In [44]:
prompt="I also have high temperature. What medicine should I take?"
start_chat(prompt)

I'm here to help—what would you like to know?
User: I also have high temperature. What medicine should I take?
 - DB CALL: execute_query(SELECT t1.medication_name FROM Diagnoses t1 JOIN Symptoms t2 ON t1.symptom_id = t2.symptom_id WHERE t2.symptom_description LIKE '%high temperature%')
Ibuprofen is the recommended medication.



In [45]:
prompt="What are the side effects of Ibuprofen?"
start_chat(prompt)

How can I assist you today?
User: What are the side effects of Ibuprofen?
 - DB CALL: execute_query(SELECT side_effects FROM Diagnoses WHERE medication_name LIKE 'Ibuprofen')
The side effects of Ibuprofen include stomach upset.



In [46]:
prompt="I am experiencing headache and blurred vision.What illness do i have and how should I treat it?"
start_chat(prompt)

How can I support your health today?
User: I am experiencing headache and blurred vision.What illness do i have and how should I treat it?
 - DB CALL: execute_query(SELECT t1.diagnosis, t1.treatment_description FROM Diagnoses t1 JOIN Symptoms t2 ON t1.symptom_id = t2.symptom_id WHERE t2.symptom_description LIKE '%headache%' AND t2.symptom_description LIKE '%blurred vision%')
Based on your symptoms, you may have High Blood Pressure. The recommended treatment is to monitor blood pressure and take prescribed medication.



# **Conclusion**



## Summary of the Project

This project presents an AI-powered healthcare assistant that leverages **Retrieval-Augmented Generation (RAG)** to provide reliable, context-aware responses to user queries related to health conditions, symptoms, and treatments. By integrating **LangChain**, **ChromaDB**, and **OpenAI’s language models**, the system retrieves relevant information from publicly available healthcare documents sourced from **NHS Inform** and generates informative responses. This approach ensures that the answers are grounded in trusted data, making the assistant both explainable and useful for general healthcare awareness.


## Use Case Value 

The assistant serves as a bridge between medical knowledge and public understanding, making healthcare information more accessible to everyday users. It can be particularly valuable for:

* Individuals seeking initial understanding of medical symptoms or conditions.
* Students and learners looking for simplified, accurate health information.
* Healthcare support systems where factual and fast retrieval of medical insights is needed.


## Next Steps?

To enhance the functionality, usability, and scalability of the assistant, the following improvements are planned:

* **User Interface Integration**: Build a frontend using Streamlit or Gradio for interactive access.
* **Multi-turn Conversations**: Add memory modules to enable contextual dialogues across multiple queries.
* **Authentication**: Add basic authentication for tracking user sessions or personalizing responses.
* **Deployment**: Host the assistant on platforms like Hugging Face Spaces or Streamlit Cloud.
* **Model Flexibility**: Allow switching between different LLM providers (e.g., OpenAI, Cohere, Hugging Face).
* **Future Expansion**: Explore adding multimodal capabilities (e.g., medical image support) for richer functionality.